In [162]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
from matplotlib.lines import Line2D
from nameparser import HumanName
import gender_guesser.detector as gender
from scipy.stats import chi2_contingency
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import ethnicolr
from ethnicolr import pred_census_ln
from collections import defaultdict
from rapidfuzz import process, fuzz
import re

In [163]:
# File Paths
absolute_path = "C:\\Users\\tykun\\\OneDrive\\Documents\\SchoolDocs\VSCodeProjects\\connectedData\\board_analysis\\"
altered_dataframes = "altered_dataframes\\"
gpt_dataframes = "gpt_dataframes\\"
graphs = "graphs\\"
scripts =  "scripts\\"
board_dataframes = "board_dataframes\\"
temporary = "temporary_data\\"
college_matching = "college_matching\\"

altered_dataframe_path = f"{absolute_path}{altered_dataframes}"
gpt_dataframe_path = f"{absolute_path}{gpt_dataframes}" 
graph_path = f"{absolute_path}{graphs}"
script_path = f"{absolute_path}{scripts}"
boards_path = f"{absolute_path}{board_dataframes}"
temporary_data_path = f"{absolute_path}{temporary}"
college_matching_path = f"{absolute_path}{college_matching}"

# Valid Years
years = ["1999", "2000", "2005", "2007", "2008", "2009", "2011", "2013", "2018"]

diversity_statistics_path = f"{altered_dataframe_path}diversity_statistics.csv"
carnegie_map = pd.read_csv(f"{college_matching_path}carnegie_map_openalex.csv")
classification_map = pd.read_csv(f"{college_matching_path}cc_download.csv")
state_path = f"{temporary_data_path}state_systems_validated.csv"   
billionaires_path = f"{temporary_data_path}billionaires_1997_2015.csv"

In [164]:

############################
# Name Cleaning / Helpers
############################

def clean_name(full_name):
    """
    Removes specified substrings and patterns from a full name.
    """
    if not isinstance(full_name, str):
        return "Unknown"

    substrings_to_remove = [
        "Rev.", "SJ", "Sister", "Brother", "Father", "OP", "The Very",
        "Sr.", "O.P.","Very Rev.", "Br.", "Dr.", "Md.", "S.J.", "Very Rev",
        "M.D.", "O.P", "S.J", "J.R", "Jr.", "Jr ", "III"
    ]
    for substring in substrings_to_remove:
        full_name = full_name.replace(substring, "")

    # Remove any capital letter followed by a period and a space (e.g., "J. ")
    full_name = re.sub(r'\b[A-Z]\. ', '', full_name)
    return full_name.strip()

def get_last_name(full_name):
    """
    Extracts the last name from a full name or returns the full name if extraction fails.
    """
    full_name = clean_name(full_name)
    if not full_name or not full_name.strip():
        return str(full_name)
    parsed = HumanName(full_name)
    return str(parsed.last) if parsed.last else str(full_name)

def get_first_name(full_name):
    """
    Extracts the first name from a full name or returns the full name if extraction fails.
    """
    full_name = clean_name(full_name)
    if not full_name or not full_name.strip():
        return str(full_name)
    parsed = HumanName(full_name)
    return str(parsed.first) if parsed.first else str(full_name)

############################
# Classification Helpers
############################

def classify_ethnicity(raw_eth):
    """
    Classifies ethnicity into 'white', 'poc', or 'unknown'.
    """
    if raw_eth == 'white':
        return 'white'
    elif raw_eth in ['black', 'hispanic', 'asian', 'aian', 'api']:
        return 'poc'
    else:
        return 'unknown'

def classify_gender(gender_str):
    """
    Classifies gender into 'male', 'female', or 'unknown'.
    """
    if gender_str in ['male', 'mostly_male']:
        return 'male'
    elif gender_str in ['female', 'mostly_female']:
        return 'female'
    else:
        return 'unknown'

############################
# Gender Predictor
############################

def pred_gender_fn(df, first_name_col):
    """
    Predicts gender based on first names and classifies it.
    """
    d = gender.Detector()
    df['gender'] = df[first_name_col].apply(lambda x: d.get_gender(x))
    df['gender'] = df['gender'].apply(classify_gender)
    return df

############################
# Ethnicity Predictor
############################

def pred_ethnicity_fn(df, last_name_col):
    """
    Predicts ethnicity based on last names using Ethnicolr's pred_census_ln.
    Returns a DataFrame with a new 'ethnicity' column in {'white', 'poc', 'unknown'}.
    """

    # Ensure valid strings in last_name_col
    df[last_name_col] = df[last_name_col].fillna("Unknown").astype(str)

    # Remove non-alphanumeric chars from each last name
    df[last_name_col] = df[last_name_col].apply(
        lambda x: ''.join(filter(str.isalnum, x)) if isinstance(x, str) else "Unknown"
    )
    # Map year to Ethnicolr-supported census year
    census_year = 2010

    # Use Ethnicolr to predict ethnicity
    df = pred_census_ln(df, last_name_col, year=census_year)
    df['race'] = df['race'].fillna('unknown')
    df['ethnicity'] = df['race'].apply(classify_ethnicity)

    # Optionally drop the raw prediction column if not needed
    df.drop(columns=['race'], inplace=True, errors='ignore')

    return df

############################
# President Classification
############################

def classify_president_genders(valid_years, altered_dataframe_path):
    """
    Classifies gender for each president in the specified years and adds 'female_president'.
    Returns a dict: Institution -> female_president (bool).
    """
    president_data = {}

    for year in valid_years:
        president_df_path = f"{altered_dataframe_path}{year}_presidents.csv"
        president_df = pd.read_csv(president_df_path)

        # Extract first names and predict gender
        president_df['first_name'] = president_df['Name'].apply(get_first_name)
        president_df['gender'] = president_df['first_name'].apply(
            lambda x: gender.Detector().get_gender(x)
        )
        president_df['gender'] = president_df['gender'].apply(classify_gender)

        # Mark female_president
        president_df['female_president'] = president_df['gender'].eq('female')

        # Store results in a dictionary by Institution
        for _, row in president_df.iterrows():
            # Overwrites if multiple rows for the same Institution
            president_data[row['Institution']] = row['female_president']

    return president_data


In [165]:
def compute_board_turnover(current_board_df, previous_board_df, years_diff):
    """
    Given a DataFrame for the current year and a DataFrame for the previous year,
    compute the normalized number of board members swapped (entered or left) for each Institution.
    
    Args:
        current_board_df (pd.DataFrame): Board data for the current year, 
                                         must have columns ['Institution', 'Name'] at minimum.
        previous_board_df (pd.DataFrame or None): Board data for the previous year,
                                                 or None if this is the first year.
        years_diff (int): Number of years between current and previous data points.
    
    Returns:
        pd.DataFrame: A DataFrame with columns ['Institution', 'board_turnover'].
                      'board_turnover' is the number of distinct members who either left
                      or joined, normalized by the number of years between data points.
    """
    # If there's no previous data (first year), turnover = 0 for all institutions
    years_diff = int(years_diff)
    if previous_board_df is None or previous_board_df.empty:
        turnover_df = current_board_df[['Institution']].drop_duplicates().copy()
        turnover_df['board_turnover'] = 0
        return turnover_df
    
    # Collect all unique institutions from both current and previous data
    institutions = pd.concat([
        current_board_df['Institution'], 
        previous_board_df['Institution']
    ]).unique()

    turnover_rows = []
    for inst in institutions:
        # Current members for the institution
        curr_members = current_board_df.loc[
            current_board_df['Institution'] == inst, 'Name'
        ].unique()
        
        # Previous members for the institution
        prev_members = previous_board_df.loc[
            previous_board_df['Institution'] == inst, 'Name'
        ].unique()
        
        # Compute symmetric difference
        curr_set = set(curr_members)
        prev_set = set(prev_members)
        swapped_members = curr_set.symmetric_difference(prev_set)
        
        # Normalize turnover by the number of years between data points
        normalized_turnover = len(swapped_members) / years_diff
        turnover_rows.append({
            'Institution': inst,
            'board_turnover': normalized_turnover
        })

    turnover_df = pd.DataFrame(turnover_rows)
    return turnover_df


In [166]:
def process_board_data():
    """
    Processes board data for each year to compute both gender and ethnicity counts using
    board_df and double_board_df. Also predicts president gender, adds 'female_president',
    'AffiliationId', and 'PrimarySample' columns. Retains original columns and adds
    membership counts, including 'total_members' and yearly changes.

    Returns:
        pd.DataFrame: A DataFrame with original columns, yearly change statistics, and added
                      president/affiliation data for both gender and ethnicity.
    """
    numbers_list = []
    prev_board_df = None
    prev_year = None
    
    for year in years:
        print(f"Processing year: {year}")

        # Load the president data for the current year
        president_df_path = f"{altered_dataframe_path}{year}_presidents.csv"
        president_df = pd.read_csv(president_df_path)

        # Extract names & predict gender
        president_df['first_name'] = president_df['Name'].apply(get_first_name)
        president_df['last_name'] = president_df['Name'].apply(get_last_name)
        president_df = pred_gender_fn(president_df, 'first_name')

        # Mark female_president
        president_df['female_president'] = president_df['gender'] == 'female'

        # Remove duplicates so each Institution is unique
        president_df = president_df.drop_duplicates(subset='Institution').reset_index(drop=True)

        # Create dictionary mapping institutions -> {female_president}
        president_data = president_df.set_index('Institution')[['female_president']].to_dict('index')

        # Load the board data for the current year
        board_df_path = f"{boards_path}{year}_boards.csv"
        board_df = pd.read_csv(board_df_path)

        # Load the double board data for the current year
        double_board_df_path = f"{boards_path}{year}_double_board.csv"
        double_board_df = pd.read_csv(double_board_df_path)


        # Function to preprocess and compute gender counts
        def preprocess_and_count_gender(df):
            print(f"Preprocessing gender counts for {len(df)} records")  # Debugging
            # Extract names & predict gender
            df['first_name'] = df['Name'].apply(get_first_name)
            df['last_name'] = df['Name'].apply(get_last_name)
            df = pred_gender_fn(df, 'first_name')

            # Check for 'PrimarySample' column
            if 'PrimarySample' in df.columns:
                primary_sample_map = df.drop_duplicates(subset='Institution').set_index('Institution')['PrimarySample']
            else:
                # If 'PrimarySample' doesn't exist, assume False for all
                primary_sample_map = pd.Series(False, index=df['Institution'].unique())

            # Group by Institution, AffiliationId, and gender
            group_cols = ['Institution', 'AffiliationId']
            gender_counts = (
                df.groupby(group_cols + ['gender'])
                  .size()
                  .unstack('gender', fill_value=0)
                  .reset_index()
            )

            # Ensure all gender columns are present
            for col in ['male', 'female', 'unknown']:
                if col not in gender_counts.columns:
                    gender_counts[col] = 0

            # Add total_members from male/female/unknown columns
            gender_counts['total_members_board'] = gender_counts[['male', 'female', 'unknown']].sum(axis=1)

            # Map PrimarySample
            gender_counts['PrimarySample'] = gender_counts['Institution'].map(primary_sample_map).fillna(False)

            return gender_counts

        # Function to preprocess and compute ethnicity counts
        def preprocess_and_count_ethnicity(df):
            print(f"Preprocessing ethnicity counts for {len(df)} records")  # Debugging
            # Extract last names & predict ethnicity
            df['last_name'] = df['Name'].apply(get_last_name)
            df = pred_ethnicity_fn(df, 'last_name')

            # Check for 'PrimarySample' column
            if 'PrimarySample' in df.columns:
                primary_sample_map = df.drop_duplicates(subset='Institution').set_index('Institution')['PrimarySample']
            else:
                # If 'PrimarySample' doesn't exist, assume False for all
                primary_sample_map = pd.Series(False, index=df['Institution'].unique())

            # Group by Institution, AffiliationId, and ethnicity
            group_cols = ['Institution', 'AffiliationId']
            ethnicity_counts = (
                df.groupby(group_cols + ['ethnicity'])
                  .size()
                  .unstack('ethnicity', fill_value=0)
                  .reset_index()
            )

            # Ensure all ethnicity columns are present
            for col in ['white', 'poc', 'unknown']:
                if col not in ethnicity_counts.columns:
                    ethnicity_counts[col] = 0

            # Add total_ethnicity from white/poc/unknown columns
            ethnicity_counts['total_ethnicity_board'] = ethnicity_counts[['white', 'poc', 'unknown']].sum(axis=1)

            # Map PrimarySample
            ethnicity_counts['PrimarySample'] = ethnicity_counts['Institution'].map(primary_sample_map).fillna(False)

            return ethnicity_counts

        # Preprocess and count for board_df (gender and ethnicity)
        board_gender_counts = preprocess_and_count_gender(board_df)
        board_gender_counts.rename(columns={
            'male': 'male_board',
            'female': 'female_board',
            'unknown': 'unknown_board',
            'total_members_board': 'total_members_board'
        }, inplace=True)

        board_ethnicity_counts = preprocess_and_count_ethnicity(board_df)
        board_ethnicity_counts.rename(columns={
            'white': 'white_board',
            'poc': 'poc_board',
            'unknown': 'unknown_eth_board',
            'total_ethnicity_board': 'total_ethnicity_board'
        }, inplace=True)

        # Preprocess and count for double_board_df (gender and ethnicity)
        double_board_gender_counts = preprocess_and_count_gender(double_board_df)
        double_board_gender_counts.rename(columns={
            'male': 'male_double',
            'female': 'female_double',
            'unknown': 'unknown_double',
            'total_members_board': 'total_members_double'
        }, inplace=True)

        double_board_ethnicity_counts = preprocess_and_count_ethnicity(double_board_df)
        double_board_ethnicity_counts.rename(columns={
            'white': 'white_double',
            'poc': 'poc_double',
            'unknown': 'unknown_eth_double',
            'total_ethnicity_board': 'total_ethnicity_double'
        }, inplace=True)

        # Merge the gender counts
        merged_gender_counts = pd.merge(
            board_gender_counts,
            double_board_gender_counts,
            on=['Institution', 'AffiliationId', 'PrimarySample'],
            how='outer'
        )

        # Merge the ethnicity counts
        merged_ethnicity_counts = pd.merge(
            board_ethnicity_counts,
            double_board_ethnicity_counts,
            on=['Institution', 'AffiliationId', 'PrimarySample'],
            how='outer'
        )

        # Now, merge gender and ethnicity counts into a single DataFrame
        merged_counts = pd.merge(
            merged_gender_counts,
            merged_ethnicity_counts,
            on=['Institution', 'AffiliationId', 'PrimarySample'],
            how='outer'
        )

        # Fill NaN values with 0 for all counts
        count_columns = [
            'male_board', 'female_board', 'unknown_board', 'total_members_board',
            'male_double', 'female_double', 'unknown_double', 'total_members_double',
            'white_board', 'poc_board', 'unknown_eth_board', 'total_ethnicity_board',
            'white_double', 'poc_double', 'unknown_eth_double', 'total_ethnicity_double'
        ]
        for col in count_columns:
            if col in merged_counts.columns:
                merged_counts[col] = merged_counts[col].fillna(0).astype(int)
            else:
                merged_counts[col] = 0

        # Define functions to compute weighted averages
        def weighted_avg(row, col_board, col_double, total_b, total_d):
            total = row[total_b] + row[total_d]
            if total == 0:
                return 0
            return round(
                (row[col_board] * row[total_b] + row[col_double] * row[total_d]) / total
            )

        # Apply weighted averages for gender counts
        merged_counts['male'] = merged_counts.apply(
            lambda row: int(weighted_avg(row, 'male_board', 'male_double',
                                       'total_members_board', 'total_members_double')),
            axis=1
        )
        merged_counts['female'] = merged_counts.apply(
            lambda row: int(weighted_avg(row, 'female_board', 'female_double',
                                       'total_members_board', 'total_members_double')),
            axis=1
        )
        merged_counts['unknown'] = merged_counts.apply(
            lambda row: int(weighted_avg(row, 'unknown_board', 'unknown_double',
                                       'total_members_board', 'total_members_double')),
            axis=1
        )
        merged_counts['total_members'] = merged_counts.apply(
            lambda row: int(row['total_members_board'] + row['total_members_double']),
            axis=1
        )

        # Apply weighted averages for ethnicity counts
        merged_counts['white'] = merged_counts.apply(
            lambda row: int(weighted_avg(row, 'white_board', 'white_double',
                                       'total_ethnicity_board', 'total_ethnicity_double')),
            axis=1
        )
        merged_counts['poc'] = merged_counts.apply(
            lambda row: int(weighted_avg(row, 'poc_board', 'poc_double',
                                       'total_ethnicity_board', 'total_ethnicity_double')),
            axis=1
        )
        merged_counts['unknown_eth'] = merged_counts.apply(
            lambda row: int(weighted_avg(row, 'unknown_eth_board', 'unknown_eth_double',
                                       'total_ethnicity_board', 'total_ethnicity_double')),
            axis=1
        )
        merged_counts['total_ethnicity'] = merged_counts.apply(
            lambda row: int(row['total_ethnicity_board'] + row['total_ethnicity_double']),
            axis=1
        )

        # Add 'Year' column
        merged_counts['Year'] = year

        # Add 'female_president' column
        merged_counts['female_president'] = merged_counts['Institution'].map(
            lambda inst: president_data.get(inst, {}).get('female_president', False)
        )

        # Reorder and select relevant columns
        main_cols = ['Year', 'Institution', 'AffiliationId', 'female_president', 'PrimarySample',
                     'total_members', 'male', 'female', 'unknown',
                     'total_ethnicity', 'white', 'poc', 'unknown_eth']
        counts_df = merged_counts[main_cols]

        if prev_board_df is not None and prev_year is not None:
            year_diff = int(year) - int(prev_year)
        else:
            year_diff = 1

        turnover_df = compute_board_turnover(
            current_board_df=board_df[['Institution', 'Name']], 
            previous_board_df=prev_board_df[['Institution', 'Name']] if prev_board_df is not None else None,
            years_diff = year_diff
        )

        counts_df = pd.merge(
            counts_df,
            turnover_df,
            on='Institution',   # We only have institution-level in the aggregated counts
            how='left'
        )
        # If any board_turnover NaN remains (shouldn't happen, but just in case):
        counts_df['board_turnover'] = counts_df['board_turnover'].fillna(0).astype(int)

        # Append to numbers list
        numbers_list.append(counts_df)
        prev_board_df = board_df.copy()
        prev_year = year

    numbers_df = pd.concat(numbers_list, ignore_index=True)

    # Ensure no missing values
    numbers_df.fillna(0, inplace=True)

    # Sort
    numbers_df.sort_values(by=['Year', 'Institution', 'AffiliationId'], inplace=True)

    # Calculate yearly changes for gender and ethnicity columns
    for col in ['male', 'female', 'unknown', 'white', 'poc', 'unknown_eth']:
        numbers_df[f"{col}_change"] = (
            numbers_df.groupby(['Institution', 'AffiliationId'])[col]
                        .diff()
                        .fillna(0)
                        .astype(int)
        )

    # Final sort
    numbers_df.sort_values(by=['Year', 'Institution', 'AffiliationId'], inplace=True)

    return numbers_df

In [167]:
university_board_statistics_df = process_board_data()
university_board_statistics_df.to_csv(altered_dataframe_path + "university_board_statistics.csv", index = False)

Processing year: 1999
Preprocessing gender counts for 11581 records


C:\Users\tykun\AppData\Local\Temp\ipykernel_7984\1221492181.py:79: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gender_counts['PrimarySample'] = gender_counts['Institution'].map(primary_sample_map).fillna(False)


Preprocessing ethnicity counts for 11581 records


C:\Users\tykun\AppData\Local\Temp\ipykernel_7984\1221492181.py:115: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ethnicity_counts['PrimarySample'] = ethnicity_counts['Institution'].map(primary_sample_map).fillna(False)


Preprocessing gender counts for 716 records
Preprocessing ethnicity counts for 716 records
Processing year: 2000
Preprocessing gender counts for 12383 records
Preprocessing ethnicity counts for 12383 records
Preprocessing gender counts for 649 records
Preprocessing ethnicity counts for 649 records
Processing year: 2005
Preprocessing gender counts for 12278 records


...

Preprocessing gender counts for 384 records
Preprocessing ethnicity counts for 384 records
Processing year: 2009
Preprocessing gender counts for 11886 records
Preprocessing ethnicity counts for 11886 records
Preprocessing gender counts for 336 records
Preprocessing ethnicity counts for 336 records
Processing year: 2011
Preprocessing gender counts for 11221 records
Preprocessing ethnicity counts for 11221 records
Preprocessing gender counts for 296 records
Preprocessing ethnicity counts for 296 records
Processing year: 2013
Preprocessing gender counts for 7833 records


C:\Users\tykun\AppData\Local\Temp\ipykernel_7984\1221492181.py:79: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gender_counts['PrimarySample'] = gender_counts['Institution'].map(primary_sample_map).fillna(False)


Preprocessing ethnicity counts for 7833 records


C:\Users\tykun\AppData\Local\Temp\ipykernel_7984\1221492181.py:115: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ethnicity_counts['PrimarySample'] = ethnicity_counts['Institution'].map(primary_sample_map).fillna(False)


Preprocessing gender counts for 280 records
Preprocessing ethnicity counts for 280 records
Processing year: 2018
Preprocessing gender counts for 7422 records
Preprocessing ethnicity counts for 7422 records
Preprocessing gender counts for 245 records
Preprocessing ethnicity counts for 245 records


In [168]:
def map_affiliation_id(affiliation_df, university_board_statistics_df):
    count_updates = 0  # Counter for the number of updates

    for index, row in affiliation_df.iterrows():
        institution = row['FullName']
        affiliation_id = row['AffiliationId']
        
        # Get the number of matching rows before updating
        matching_rows = university_board_statistics_df[
            (university_board_statistics_df['Institution'] == institution) & 
            (university_board_statistics_df['AffiliationId'].isna())
        ]
        
        # Update AffiliationId if matches are found
        if not matching_rows.empty:
            count_updates += len(matching_rows)
            university_board_statistics_df.loc[
                (university_board_statistics_df['Institution'] == institution) & 
                (university_board_statistics_df['AffiliationId'].isna()), 'AffiliationId'
            ] = affiliation_id

    # Print the total number of updates made
    print(f"Total AffiliationId entries updated: {count_updates}")

    return university_board_statistics_df

In [169]:
#combined a couple different scripts here so the variable names are slightly changed and makes code a bit messy 


university_board_statistics = university_board_statistics_df
state_system_df = pd.read_csv(state_path)

# Set default value of StateSystem to False
university_board_statistics['StateSystem'] = False

for index, row in state_system_df.iterrows():
    if pd.notna(row['StateSystem']):
        institution = row['Institution']
        affiliation_id = row['AffiliationId']
        state_system_value = row['StateSystem']
        
        university_board_statistics.loc[
            (university_board_statistics['Institution'] == institution) | 
            (university_board_statistics['AffiliationId'] == affiliation_id), 
            'StateSystem'
        ] = state_system_value

university_board_statistics.to_csv(f"{altered_dataframe_path}sample_board_statistics.csv", index=False)

C:\Users\tykun\AppData\Local\Temp\ipykernel_7984\2976693132.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Arizona Board of Regents' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  university_board_statistics.loc[


In [170]:
for year in years:
    # Load the data for the current year
    boards_df = pd.read_csv(f"{boards_path}{year}_boards.csv")
    
    # Group by Institution and aggregate unique Name values into sets
    grouped = boards_df.groupby('Institution')['Name'].apply(set).reset_index()

    # Track processed institutions and new state system rows
    processed_institutions = set()
    new_state_system_rows = []

    # Create a mapping of name sets (as tuples) to lists of institutions that share them
    name_to_institutions = {}

    for i, group in grouped.iterrows():
        institution = group['Institution']
        names_set = tuple(group['Name'])  # Convert set to a tuple to make it hashable
        
        # Group institutions by their name sets
        if names_set not in name_to_institutions:
            name_to_institutions[names_set] = []
        name_to_institutions[names_set].append(institution)

    # Process each group of institutions that share the same name set
    for names_set, institutions in name_to_institutions.items():
        if len(institutions) > 1:  # Only consider groups with duplicates
            processed_institutions.update(institutions)
            
            # Check if any of the institutions have a state system value
            is_state_system = university_board_statistics.loc[
                (university_board_statistics['Institution'].isin(institutions)) & 
                (university_board_statistics['StateSystem'].notna()) &
                (university_board_statistics['StateSystem'] != False)
            ].any().any()

            if is_state_system:
                # Extract the StateSystem value
                state_system_name = university_board_statistics.loc[
                    (university_board_statistics['Institution'].isin(institutions)) & 
                    (university_board_statistics['StateSystem'].notna()) &
                    (university_board_statistics['StateSystem'] != False), 'StateSystem'
                ].values[0]

                # Use one of the institutions' diversity statistics
                source_stats = university_board_statistics.loc[
                    (university_board_statistics['Institution'].isin(institutions)) & 
                    (university_board_statistics['Year'] == int(year))
                ]

                if not source_stats.empty:
                    source_stats = source_stats.iloc[0]
                    new_row = {
                        'Year': int(year),
                        'Institution': state_system_name,
                        'carnegie_id': np.nan,
                        'AffiliationId': np.nan,
                        'female_president': source_stats['female_president'],
                        'female': source_stats['female'],
                        'male': source_stats['male'],
                        'board_turnover': source_stats['board_turnover'],
                        'unknown': source_stats['unknown'],
                        'unknown_change': source_stats['unknown_change'],
                        'male_change': source_stats['male_change'],
                        'female_change': source_stats['female_change'],
                        'white': source_stats['white'],
                        'poc': source_stats['poc'],
                        'unknown_eth': source_stats['unknown_eth'],
                        'white_change': source_stats['white_change'],
                        'poc_change': source_stats['poc_change'],
                        'unknown_eth_change': source_stats['unknown_eth_change'],
                        'StateSystem': True
                    }
                    new_state_system_rows.append(new_row)

    # Remove duplicate entries from the university statistics
    university_board_statistics = university_board_statistics[
        ~((university_board_statistics['Institution'].isin(processed_institutions)) & 
          (university_board_statistics['Year'] == int(year)))
    ]

    # Append new state system rows, if any
    if new_state_system_rows:
        new_rows_df = pd.DataFrame(new_state_system_rows)
        university_board_statistics = pd.concat([university_board_statistics, new_rows_df], ignore_index=True)
        print(f"Replaced {len(processed_institutions)} duplicate entries with state system rows for year {year}.")

# Save the updated dataframe to CSV
university_board_statistics = university_board_statistics.sort_values(by = ["Year", "Institution"])
university_board_statistics.to_csv(f"{altered_dataframe_path}sample_board_statistics.csv", index=False)
print("Updated university_board_statistics saved.")


Updated university_board_statistics saved.


In [171]:
# -- Refactored Code --
board_statistics = pd.read_csv(f"{altered_dataframe_path}sample_board_statistics.csv")


# 1. Remove any existing 'carnegie_id', 'state', 'control' columns
board_statistics.drop(columns=['carnegie_id', 'state', 'control'], errors='ignore', inplace=True)

# 2. Merge 'carnegie_id' from carnegie_map based on 'AffiliationId'
board_statistics = board_statistics.merge(
    carnegie_map[['AffiliationId', 'carnegie_id']], 
    on='AffiliationId', 
    how='left'
)

# 3. Merge 'state' and 'control' from classification_map based on 'carnegie_id'
board_statistics = board_statistics.merge(
    classification_map[['unitid', 'state', 'control']], 
    left_on='carnegie_id', 
    right_on='unitid', 
    how='left', 
    suffixes=('', '_classification')
)

# 4. Drop the redundant 'unitid' column
board_statistics.drop(columns='unitid', inplace=True)

# 5. Rename columns if suffixes were added
board_statistics.rename(
    columns={'state_classification': 'state', 'control_classification': 'control'},
    inplace=True
)

# 6. Remove any columns that start with "Unnamed"
board_statistics = board_statistics.loc[:, ~board_statistics.columns.str.contains('^Unnamed')]

# Final DataFrame Verification
print("\nMerged board_statistics DataFrame:")
print(board_statistics)

# Check for (and confirm removal of) unnamed columns
unnamed_columns = [col for col in board_statistics.columns if 'Unnamed' in col]
if unnamed_columns:
    print("\nUnnamed Columns Detected and Removed:")
    print(unnamed_columns)
else:
    print("\nNo Unnamed Columns Detected.")


board_statistics['control'] = board_statistics['control'].apply(
    lambda x: 'Private' if 'Private' in str(x) else 'Public' if 'Public' in str(x) else x
)



Merged board_statistics DataFrame:
      Year                      Institution  AffiliationId  female_president  \
0     1999     Abilene Christian University     60205797.0             False   
1     1999               Adelphi University     71965598.0             False   
2     1999              Agnes Scott College     64506506.0              True   
3     1999                   Albion College     45644089.0             False   
4     1999                Alfred University     49502546.0             False   
...    ...                              ...            ...               ...   
3629  2018  Worcester Polytechnic Institute    107077323.0              True   
3630  2018                Xavier University    194120229.0             False   
3631  2018   Xavier University Of Louisiana    169251466.0             False   
3632  2018                  Yale University     32971472.0             False   
3633  2018               Yeshiva University     19772626.0             False   

   

In [172]:
state_to_region = {
    # Northeast
    "CT": "Northeast", "ME": "Northeast", "MA": "Northeast", "NH": "Northeast",
    "RI": "Northeast", "VT": "Northeast", "NJ": "Northeast", "NY": "Northeast", 
    "PA": "Northeast", "DC": "Northeast",
    
    # Midwest
    "IL": "Midwest", "IN": "Midwest", "IA": "Midwest", "KS": "Midwest",
    "MI": "Midwest", "MN": "Midwest", "MO": "Midwest", "NE": "Midwest", 
    "ND": "Midwest", "OH": "Midwest", "SD": "Midwest", "WI": "Midwest",
    
    # South
    "AL": "South", "AR": "South", "DE": "South", "FL": "South",
    "GA": "South", "KY": "South", "LA": "South", "MD": "South",
    "MS": "South", "NC": "South", "OK": "South", "SC": "South",
    "TN": "South", "TX": "South", "VA": "South", "WV": "South",
    
    # West
    "AK": "West", "AZ": "West", "CA": "West", "CO": "West",
    "HI": "West", "ID": "West", "MT": "West", "NV": "West",
    "NM": "West", "OR": "West", "UT": "West", "WA": "West",
    "WY": "West"
}
# board_statistics["regoio"]
board_statistics['region'] = board_statistics['state'].map(state_to_region)
board_statistics.to_csv(f"{altered_dataframe_path}sample_board_statistics.csv", index=False)


In [173]:
all_board_members = []
for year in years:
    board_df_path = f"{boards_path}{year}_boards.csv"
    df = pd.read_csv(board_df_path)
    all_board_members.append(df)

all_board_members_df = pd.concat(all_board_members, ignore_index=True)
all_board_members_df = all_board_members_df.drop_duplicates()

all_board_names = all_board_members_df["Name"].values

In [174]:
billionaire_df = pd.read_csv(billionaires_path)
billionaire_names = billionaire_df["full_name"].values
billionaire_names = set([value.replace("& family", "") for value in billionaire_names])


In [175]:
university_boards_statistics_path = f"{altered_dataframe_path}sample_board_statistics.csv"

similarity_threshold = 90

# Preprocess billionaire names
billionaire_names = [name.strip().lower() for name in billionaire_names]

# Load university_board_statistics
university_board_statistics = pd.read_csv(university_boards_statistics_path)

university_board_statistics['num_billionaires'] = 0


# Create a temporary normalized column for matching
university_board_statistics['Institution_normalized'] = university_board_statistics['Institution'].str.strip().str.lower()
university_board_statistics['Year'] = university_board_statistics['Year'].astype(int)

# Ensure 'num_billionaires' is numeric
university_board_statistics['num_billionaires'] = pd.to_numeric(
    university_board_statistics['num_billionaires'], errors='coerce').fillna(0).astype(int)

billionaire_board_members = []

for year in years:
    print(f"\nProcessing year: {year}")
    board_df_path = f"{boards_path}{year}_boards.csv"

    # Check if the board file exists
    try:
        board_df = pd.read_csv(board_df_path)
        print(f"Loaded board data for year {year}: {board_df.shape[0]} records.")
    except FileNotFoundError:
        print(f"Error: File {board_df_path} does not exist. Skipping year {year}.")
        continue

    # Normalize 'Name' and 'Institution' in board_df
    board_df['Name'] = board_df['Name'].str.strip().str.lower()
    board_df['Institution_normalized'] = board_df['Institution'].str.strip().str.lower()

    # Create a lookup dictionary for institution by name
    name_to_institution = dict(zip(board_df['Name'], board_df['Institution_normalized']))

    # Get all board member names
    all_board_names = board_df['Name'].tolist()

    counts_per_institution = defaultdict(int)

    # Match each board member to billionaire names
    for member in all_board_names:
        result = process.extractOne(member, billionaire_names, scorer=fuzz.ratio, score_cutoff=similarity_threshold)
        if result is not None:
            match, score, _ = result
            billionaire_board_members.append((member, match, score, year))
            
            institution_name = name_to_institution.get(member)
            if institution_name:
                counts_per_institution[institution_name] += 1

    print(counts_per_institution)

    # Instead of using masks, we do direct lookups.
    for institution, count in counts_per_institution.items():
        # Flag to check if we found a match
        found_match = False

        # Iterate over each row and compare year and institution directly
        for idx in university_board_statistics.index:
            row_year = university_board_statistics.at[idx, 'Year']
            row_institution = university_board_statistics.at[idx, 'Institution_normalized']

            if str(row_year) == str(year) and str(row_institution) == str(institution):
                # Increment num_billionaires for this row
                current_value = university_board_statistics.at[idx, 'num_billionaires']
                university_board_statistics.at[idx, 'num_billionaires'] = current_value + count
                found_match = True

        if not found_match:
            print(f"Warning: No matching rows found for Year: {year}, Institution: {institution}")
        else:
            print(f"Updated rows for Year: {year}, Institution: {institution} with count {count}")

    counts_per_institution.clear()

# Drop the temporary normalized column
university_board_statistics.drop(columns=['Institution_normalized'], inplace=True)

# Final Summary
print(f"\nTotal number of board members who are billionaires (similarity ≥ {similarity_threshold}%): {len(billionaire_board_members)}")
for board_member, matched_billionaire, similarity, year in billionaire_board_members:
    print(f"Year: {year}, Board Member: {board_member}, Matched Billionaire: {matched_billionaire}, Similarity: {similarity}%")

# Save the updated statistics
university_board_statistics.to_csv(university_boards_statistics_path, index=False)
print("\nUpdated university_board_statistics saved successfully.")



Processing year: 1999
Loaded board data for year 1999: 11581 records.
defaultdict(<class 'int'>, {'abilene christian university': 1, 'bard college': 1, 'california institute of technology': 2, 'carnegie mellon university': 1, 'centenary college of louisiana': 1, 'centre college': 1, 'chapman university': 1, 'claremont mckenna college': 1, 'college of wooster': 1, 'cornell university': 1, 'creighton university': 2, 'emerson college': 1, 'emory university': 1, 'george washington university': 1, 'new york university': 4, 'georgetown university': 1, 'gonzaga university': 1, 'grinnell college': 1, 'hardin simmons university': 1, 'institute for advanced study': 2, 'johns hopkins university': 3, 'juilliard school': 1, 'lafayette college': 1, 'lawrence university': 1, 'loma linda university': 1, 'massachusetts institute of technology': 1, 'miami university': 1, 'mississippi college': 1, 'oral roberts university': 1, 'reed college': 1, 'rochester institute of technology': 1, 'stanford universi

In [176]:
def map_affiliation_id(affiliation_df, university_board_statistics_df):
    count_updates = 0  # Counter for the number of updates

    for index, row in affiliation_df.iterrows():
        institution = row['FullName']
        affiliation_id = row['AffiliationId']
        
        # Get the number of matching rows before updating
        matching_rows = university_board_statistics_df[
            (university_board_statistics_df['Institution'] == institution) & 
            (university_board_statistics_df['AffiliationId'].isna())
        ]
        
        # Update AffiliationId if matches are found
        if not matching_rows.empty:
            count_updates += len(matching_rows)
            university_board_statistics_df.loc[
                (university_board_statistics_df['Institution'] == institution) & 
                (university_board_statistics_df['AffiliationId'].isna()), 'AffiliationId'
            ] = affiliation_id

    # Print the total number of updates made
    print(f"Total AffiliationId entries updated: {count_updates}")

    return university_board_statistics_df

In [177]:
#combined a couple different scripts here so the variable names are slightly changed and makes code a bit messy 


university_board_statistics = university_board_statistics_df
state_system_df = pd.read_csv(state_path)

# Set default value of StateSystem to False
university_board_statistics['StateSystem'] = False

for index, row in state_system_df.iterrows():
    if pd.notna(row['StateSystem']):
        institution = row['Institution']
        affiliation_id = row['AffiliationId']
        state_system_value = row['StateSystem']
        
        university_board_statistics.loc[
            (university_board_statistics['Institution'] == institution) | 
            (university_board_statistics['AffiliationId'] == affiliation_id), 
            'StateSystem'
        ] = state_system_value

university_board_statistics.to_csv(f"{altered_dataframe_path}sample_board_statistics.csv", index=False)

C:\Users\tykun\AppData\Local\Temp\ipykernel_7984\2976693132.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Arizona Board of Regents' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  university_board_statistics.loc[


In [178]:
for year in years:
    # Load the data for the current year
    boards_df = pd.read_csv(f"{boards_path}{year}_boards.csv")
    
    # Group by Institution and aggregate unique Name values into sets
    grouped = boards_df.groupby('Institution')['Name'].apply(set).reset_index()

    # Track processed institutions and new state system rows
    processed_institutions = set()
    new_state_system_rows = []

    # Create a mapping of name sets (as tuples) to lists of institutions that share them
    name_to_institutions = {}

    for i, group in grouped.iterrows():
        institution = group['Institution']
        names_set = tuple(group['Name'])  # Convert set to a tuple to make it hashable
        
        # Group institutions by their name sets
        if names_set not in name_to_institutions:
            name_to_institutions[names_set] = []
        name_to_institutions[names_set].append(institution)

    # Process each group of institutions that share the same name set
    for names_set, institutions in name_to_institutions.items():
        if len(institutions) > 1:  # Only consider groups with duplicates
            processed_institutions.update(institutions)
            
            # Check if any of the institutions have a state system value
            is_state_system = university_board_statistics.loc[
                (university_board_statistics['Institution'].isin(institutions)) & 
                (university_board_statistics['StateSystem'].notna()) &
                (university_board_statistics['StateSystem'] != False)
            ].any().any()

            if is_state_system:
                # Extract the StateSystem value
                state_system_name = university_board_statistics.loc[
                    (university_board_statistics['Institution'].isin(institutions)) & 
                    (university_board_statistics['StateSystem'].notna()) &
                    (university_board_statistics['StateSystem'] != False), 'StateSystem'
                ].values[0]

                # Use one of the institutions' diversity statistics
                source_stats = university_board_statistics.loc[
                    (university_board_statistics['Institution'].isin(institutions)) & 
                    (university_board_statistics['Year'] == int(year))
                ]

                if not source_stats.empty:
                    source_stats = source_stats.iloc[0]
                    new_row = {
                        'Year': int(year),
                        'Institution': state_system_name,
                        'carnegie_id': np.nan,
                        'AffiliationId': np.nan,
                        'female_president': source_stats['female_president'],
                        # 'poc_president': source_stats['poc_president'],
                        # 'poc': source_stats['poc'],
                        # 'white': source_stats['white'],
                        'female': source_stats['female'],
                        'male': source_stats['male'],
                        'unknown': source_stats['unknown'],
                        # 'poc_change': source_stats['poc_change'],
                        # 'white_change': source_stats['white_change'],
                        'unknown_change': source_stats['unknown_change'],
                        'male_change': source_stats['male_change'],
                        'female_change': source_stats['female_change'],
                        'StateSystem': True
                    }
                    new_state_system_rows.append(new_row)

    # Remove duplicate entries from the university statistics
    university_board_statistics = university_board_statistics[
        ~((university_board_statistics['Institution'].isin(processed_institutions)) & 
          (university_board_statistics['Year'] == int(year)))
    ]

    # Append new state system rows, if any
    if new_state_system_rows:
        new_rows_df = pd.DataFrame(new_state_system_rows)
        university_board_statistics = pd.concat([university_board_statistics, new_rows_df], ignore_index=True)
        print(f"Replaced {len(processed_institutions)} duplicate entries with state system rows for year {year}.")

# Save the updated dataframe to CSV
university_board_statistics = university_board_statistics.sort_values(by = ["Year", "Institution"])
university_board_statistics.to_csv(f"{altered_dataframe_path}sample_board_statistics.csv", index=False)
print("Updated university_board_statistics saved.")


Updated university_board_statistics saved.


In [179]:
# -- Refactored Code --
board_statistics = pd.read_csv(f"{altered_dataframe_path}sample_board_statistics.csv")


# 1. Remove any existing 'carnegie_id', 'state', 'control' columns
board_statistics.drop(columns=['carnegie_id', 'state', 'control'], errors='ignore', inplace=True)

# 2. Merge 'carnegie_id' from carnegie_map based on 'AffiliationId'
board_statistics = board_statistics.merge(
    carnegie_map[['AffiliationId', 'carnegie_id']], 
    on='AffiliationId', 
    how='left'
)

# 3. Merge 'state' and 'control' from classification_map based on 'carnegie_id'
board_statistics = board_statistics.merge(
    classification_map[['unitid', 'state', 'control']], 
    left_on='carnegie_id', 
    right_on='unitid', 
    how='left', 
    suffixes=('', '_classification')
)

# 4. Drop the redundant 'unitid' column
board_statistics.drop(columns='unitid', inplace=True)

# 5. Rename columns if suffixes were added
board_statistics.rename(
    columns={'state_classification': 'state', 'control_classification': 'control'},
    inplace=True
)

# 6. Remove any columns that start with "Unnamed"
board_statistics = board_statistics.loc[:, ~board_statistics.columns.str.contains('^Unnamed')]

# Final DataFrame Verification
print("\nMerged board_statistics DataFrame:")
print(board_statistics)

# Check for (and confirm removal of) unnamed columns
unnamed_columns = [col for col in board_statistics.columns if 'Unnamed' in col]
if unnamed_columns:
    print("\nUnnamed Columns Detected and Removed:")
    print(unnamed_columns)
else:
    print("\nNo Unnamed Columns Detected.")


board_statistics['control'] = board_statistics['control'].apply(
    lambda x: 'Private' if 'Private' in str(x) else 'Public' if 'Public' in str(x) else x
)



Merged board_statistics DataFrame:
      Year                      Institution  AffiliationId  female_president  \
0     1999     Abilene Christian University     60205797.0             False   
1     1999               Adelphi University     71965598.0             False   
2     1999              Agnes Scott College     64506506.0              True   
3     1999                   Albion College     45644089.0             False   
4     1999                Alfred University     49502546.0             False   
...    ...                              ...            ...               ...   
3629  2018  Worcester Polytechnic Institute    107077323.0              True   
3630  2018                Xavier University    194120229.0             False   
3631  2018   Xavier University Of Louisiana    169251466.0             False   
3632  2018                  Yale University     32971472.0             False   
3633  2018               Yeshiva University     19772626.0             False   

   

In [180]:
state_to_region = {
    # Northeast
    "CT": "Northeast", "ME": "Northeast", "MA": "Northeast", "NH": "Northeast",
    "RI": "Northeast", "VT": "Northeast", "NJ": "Northeast", "NY": "Northeast", 
    "PA": "Northeast", "DC": "Northeast",
    
    # Midwest
    "IL": "Midwest", "IN": "Midwest", "IA": "Midwest", "KS": "Midwest",
    "MI": "Midwest", "MN": "Midwest", "MO": "Midwest", "NE": "Midwest", 
    "ND": "Midwest", "OH": "Midwest", "SD": "Midwest", "WI": "Midwest",
    
    # South
    "AL": "South", "AR": "South", "DE": "South", "FL": "South",
    "GA": "South", "KY": "South", "LA": "South", "MD": "South",
    "MS": "South", "NC": "South", "OK": "South", "SC": "South",
    "TN": "South", "TX": "South", "VA": "South", "WV": "South",
    
    # West
    "AK": "West", "AZ": "West", "CA": "West", "CO": "West",
    "HI": "West", "ID": "West", "MT": "West", "NV": "West",
    "NM": "West", "OR": "West", "UT": "West", "WA": "West",
    "WY": "West"
}
# board_statistics["regoio"]
board_statistics['region'] = board_statistics['state'].map(state_to_region)
board_statistics.to_csv(f"{altered_dataframe_path}sample_board_statistics.csv", index=False)


In [181]:
all_board_members = []
for year in years:
    board_df_path = f"{boards_path}{year}_boards.csv"
    df = pd.read_csv(board_df_path)
    all_board_members.append(df)

all_board_members_df = pd.concat(all_board_members, ignore_index=True)
all_board_members_df = all_board_members_df.drop_duplicates()

all_board_names = all_board_members_df["Name"].values

In [182]:
billionaire_df = pd.read_csv(billionaires_path)
billionaire_names = billionaire_df["full_name"].values
billionaire_names = set([value.replace("& family", "") for value in billionaire_names])


In [183]:
university_boards_statistics_path = f"{altered_dataframe_path}sample_board_statistics.csv"

similarity_threshold = 90

# Preprocess billionaire names
billionaire_names = [name.strip().lower() for name in billionaire_names]

# Load university_board_statistics
university_board_statistics = pd.read_csv(university_boards_statistics_path)

university_board_statistics['num_billionaires'] = 0


# Create a temporary normalized column for matching
university_board_statistics['Institution_normalized'] = university_board_statistics['Institution'].str.strip().str.lower()
university_board_statistics['Year'] = university_board_statistics['Year'].astype(int)

# Ensure 'num_billionaires' is numeric
university_board_statistics['num_billionaires'] = pd.to_numeric(
    university_board_statistics['num_billionaires'], errors='coerce').fillna(0).astype(int)

billionaire_board_members = []

for year in years:
    print(f"\nProcessing year: {year}")
    board_df_path = f"{boards_path}{year}_boards.csv"

    # Check if the board file exists
    try:
        board_df = pd.read_csv(board_df_path)
        print(f"Loaded board data for year {year}: {board_df.shape[0]} records.")
    except FileNotFoundError:
        print(f"Error: File {board_df_path} does not exist. Skipping year {year}.")
        continue

    # Normalize 'Name' and 'Institution' in board_df
    board_df['Name'] = board_df['Name'].str.strip().str.lower()
    board_df['Institution_normalized'] = board_df['Institution'].str.strip().str.lower()

    # Create a lookup dictionary for institution by name
    name_to_institution = dict(zip(board_df['Name'], board_df['Institution_normalized']))

    # Get all board member names
    all_board_names = board_df['Name'].tolist()

    counts_per_institution = defaultdict(int)

    # Match each board member to billionaire names
    for member in all_board_names:
        result = process.extractOne(member, billionaire_names, scorer=fuzz.ratio, score_cutoff=similarity_threshold)
        if result is not None:
            match, score, _ = result
            billionaire_board_members.append((member, match, score, year))
            
            institution_name = name_to_institution.get(member)
            if institution_name:
                counts_per_institution[institution_name] += 1

    print(counts_per_institution)

    # Instead of using masks, we do direct lookups.
    for institution, count in counts_per_institution.items():
        # Flag to check if we found a match
        found_match = False

        # Iterate over each row and compare year and institution directly
        for idx in university_board_statistics.index:
            row_year = university_board_statistics.at[idx, 'Year']
            row_institution = university_board_statistics.at[idx, 'Institution_normalized']

            if str(row_year) == str(year) and str(row_institution) == str(institution):
                # Increment num_billionaires for this row
                current_value = university_board_statistics.at[idx, 'num_billionaires']
                university_board_statistics.at[idx, 'num_billionaires'] = current_value + count
                found_match = True

        if not found_match:
            print(f"Warning: No matching rows found for Year: {year}, Institution: {institution}")
        else:
            print(f"Updated rows for Year: {year}, Institution: {institution} with count {count}")

    counts_per_institution.clear()

# Drop the temporary normalized column
university_board_statistics.drop(columns=['Institution_normalized'], inplace=True)

# Final Summary
print(f"\nTotal number of board members who are billionaires (similarity ≥ {similarity_threshold}%): {len(billionaire_board_members)}")
for board_member, matched_billionaire, similarity, year in billionaire_board_members:
    print(f"Year: {year}, Board Member: {board_member}, Matched Billionaire: {matched_billionaire}, Similarity: {similarity}%")

# Save the updated statistics
university_board_statistics.to_csv(university_boards_statistics_path, index=False)
print("\nUpdated university_board_statistics saved successfully.")



Processing year: 1999
Loaded board data for year 1999: 11581 records.
defaultdict(<class 'int'>, {'abilene christian university': 1, 'bard college': 1, 'california institute of technology': 2, 'carnegie mellon university': 1, 'centenary college of louisiana': 1, 'centre college': 1, 'chapman university': 1, 'claremont mckenna college': 1, 'college of wooster': 1, 'cornell university': 1, 'creighton university': 2, 'emerson college': 1, 'emory university': 1, 'george washington university': 1, 'new york university': 4, 'georgetown university': 1, 'gonzaga university': 1, 'grinnell college': 1, 'hardin simmons university': 1, 'institute for advanced study': 2, 'johns hopkins university': 3, 'juilliard school': 1, 'lafayette college': 1, 'lawrence university': 1, 'loma linda university': 1, 'massachusetts institute of technology': 1, 'miami university': 1, 'mississippi college': 1, 'oral roberts university': 1, 'reed college': 1, 'rochester institute of technology': 1, 'stanford universi